# Cleaning the ECE dataset

The ECE (Censal Evaluation of Students, in its Spanish acronym) is the national standardized student test conducted by the Ministry of Education of Peru. In this notebook, I will clean the ECE dataset corresponding to the average 2016 Math scores by school for the students in the second year of primary education in public schools in Lima.

This dataset comes from a formal request of access to public data I made to the Ministry of Education of Peru in 2018, when I was still living in Lima.

I'll start by importing the libraries I'll use.

In [1]:
import pandas as pd
import re

Taking a first glance at the data:

In [2]:
data_file = '../../data/raw/IE 2P ECE 16_Lima.xlsx'
df = pd.read_excel(data_file,
                   sheet_name = 'IE 2P ECE 16',
                   header = 1,
                   skipfooter = 7,
                   usecols = [0, 2, 7, 15, 16, 23])
df.head()

,Código Modular,Nombre de I.E.,Código geográfico,Cantidad de alumnos evaluados,Cobertura (%),Medida promedio de la IE equiparada con 2007
0,207795,5040 PEDRO RUIZ,70101,38,90.476190,624.540697
1,207803,5041,70103,32,74.418605,672.498894
2,207845,5031 CESAR VALLEJO,70101,53,94.642857,581.662211
3,207852,5032 ENRIQUE DEL HORME,70101,52,96.296296,590.868747
4,207894,5019 - AUGUSTO CAZORLA,70101,68,100.000000,564.494987


Every data point in this dataframe is a public primary school in Lima Metropolina. These are the columns I've exported:

* Codigo modular (official unique school ID)
* Nombre de IE (School name)
* Codigo geografico (ID of the district where the school is)
* Cantidad de alumnos evaluados (number of students who took the test)
* Cobertura (percentage of students who took test among all the students in second grade in that school)
* Medida promedio (average test score, calibrated to the 2007 test so it is potentially comparable with the results of other years)

Now I'll rename the columns according to the data they represent:

In [3]:
rename_dict = {'Código Modular': 'school_id',
               'Nombre de I.E.': 'school_name',
               'Código geográfico': 'IDDIST',
               'Cantidad de alumnos evaluados': 'num_test_takers',
               'Cobertura (%)': 'test_coverage',
               'Medida promedio de la IE equiparada con 2007': 'average_score'}
df = df.rename(columns = rename_dict)

Finally, I'll need the variable `IDDIST` to be in string format and to contain six characters in all instances.

In [4]:
df['IDDIST'] = df['IDDIST'].astype('str')
df['IDDIST'].loc[df['IDDIST'].apply(lambda x: x[0] == '7')] = '0' + df['IDDIST'][df['IDDIST'].apply(lambda x: x[0] == '7')]

/home/luis_eduardo/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Looking at this new dataset:

In [5]:
df.head()

,school_id,school_name,IDDIST,num_test_takers,test_coverage,average_score
0,207795,5040 PEDRO RUIZ,070101,38,90.476190,624.540697
1,207803,5041,070103,32,74.418605,672.498894
2,207845,5031 CESAR VALLEJO,070101,53,94.642857,581.662211
3,207852,5032 ENRIQUE DEL HORME,070101,52,96.296296,590.868747
4,207894,5019 - AUGUSTO CAZORLA,070101,68,100.000000,564.494987


## Merging with the zones

In [6]:
zones_file = '../../data/clean/iddist_zone_lima.csv'
zones = pd.read_csv(zones_file, dtype='str')

In [7]:
df = pd.merge(df, zones, how='inner', on='IDDIST')
df.head()

,school_id,school_name,IDDIST,num_test_takers,test_coverage,average_score,zone
0,207795,5040 PEDRO RUIZ,070101,38,90.476190,624.540697,Port
1,207845,5031 CESAR VALLEJO,070101,53,94.642857,581.662211,Port
2,207852,5032 ENRIQUE DEL HORME,070101,52,96.296296,590.868747,Port
3,207894,5019 - AUGUSTO CAZORLA,070101,68,100.000000,564.494987,Port
4,207985,5023 ABELARDO GAMARRA,070101,77,98.717949,636.268876,Port


## Exporting

This is pretty much how I need my data to be for now, so I'll just export this result.

In [8]:
output_file = '../../data/clean/ECE_by_school_2p_lima_2016.csv'
df.to_csv(output_file, index=False)